In [1]:
import nbformat
from nbformat.v4 import new_notebook, new_code_cell, new_markdown_cell

# Example data: list of tuples (code, output) or just code as strings for simplicity
cells_data = [
    ("# This is a sample code cell", "# This is the output as markdown"),
    ("print('Hello, world!')", "Hello, world!"),
]

nb = new_notebook()

for code, output in cells_data:
    # Add a code cell
    nb['cells'].append(new_code_cell(code))
    # Add its corresponding output as a markdown cell (for simplicity)
    nb['cells'].append(new_markdown_cell(output))

# Specify the nbformat versions
nb['nbformat'] = 4
nb['nbformat_minor'] = 2

# Write the notebook to a file
with open('example_notebook.ipynb', 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

print("Notebook created successfully.")

Notebook created successfully.


In [1]:
import pickle as pkl
messages = pkl.load(open("customer_history_chat_history.pkl", "rb"))

In [5]:
messages

[{'role': 'system',
  'content': 'You are an intelligent assistent with access to a running ipython kernel. \nYou can use `python_code_exec` to execute python code to solve computational problems and return the output. \nYou can also use `inspect_variable` to inspect the state of the kernel by getting the value of a variable. \nYou should use `seek_human_input` when analysis finishes. When you stuck, use this function to ask human for clarification, instruction.\nThese functions execute code in local terminal with no cost, you can use them as many times as you want.\nWhen facing complex problems, you can divide them into smaller problems and run code to solve smaller ones, check the returned results and then solve the more complex one.\n'},
 {'role': 'user',
  'content': 'Here is a table description \nBackground:\n\nThis dataset comprises essential information related to airline bookings, providing insights into various aspects of the passenger booking experience. \nThe dataset is desi

In [29]:

import nbformat
from nbformat.v4 import new_notebook, new_code_cell, new_markdown_cell
import sys
sys.path.append("..")
from auto_analytics.utils.json_utils import parse_partial_json

# Example data: list of tuples (code, output) or just code as strings for simplicity
cells_data = [
    ("# This is a sample code cell", "# This is the output as markdown"),
    ("print('Hello, world!')", "Hello, world!"),
]

nb = new_notebook()


for message in messages:
    message = dict(message)
    if message.get("role") == 'system':
        nb['cells'].append(new_markdown_cell("**System**: "+message["content"]))
        # print(message.get("content"))
    elif message.get("role") == 'user':
        nb['cells'].append(new_markdown_cell("**User**: "+message["content"]))
        # print(message.get("content"))
    elif message.get("role") == 'assistant':
        if message["content"]:
            nb['cells'].append(new_markdown_cell("**Assistant**: "+message["content"]))
        if message['tool_calls']:
            for tool_call in message['tool_calls']:
                
                function_name = tool_call.function.name
                function_args = parse_partial_json(tool_call.function.arguments)
                if function_name == "python_code_exec" and function_args.get("code"):
                    nb['cells'].append(new_code_cell(function_args["code"]))
                # print(function_args["code"])
                # nb['cells'].append(new_markdown_cell("**Tool call**: "+tool_call["tool_name"]))
                # tool_call
                # for tool_output in tool_call['tool_outputs']:
                    # nb['cells'].append(new_markdown_cell("**Tool output**: "+tool_output["content"]))
    elif message.get("role") == 'tool':
        nb['cells'].append(new_markdown_cell("**Tool output**: "+message["content"]))

# Specify the nbformat versions
nb['nbformat'] = 4
nb['nbformat_minor'] = 2

# Write the notebook to a file
with open('../reports/example_chat_report.ipynb', 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

print("Notebook created successfully.")


Notebook created successfully.


In [30]:
from auto_analytics.tool_chat_loop import python_code_exec

In [33]:
out, captured, disp_images = python_code_exec("print('Hello, world!')")

### Figure capturing and saving

In [ ]:
import nbformat
from nbformat.v4 import new_notebook, new_code_cell, new_output


code = "print('welcome!');import matplotlib.pyplot as plt\nplt.plot([1,2,3,4])\nplt.show()"
out, captured, disp_images = python_code_exec(code)
# encoded_img_data = captured.outputs[0].data['image/png']

# Create a new notebook
nb = new_notebook()
# Add a code cell with the executed code
code_cell = new_code_cell(source=code)
output_stdout = new_output(output_type="stream",
            name="stdout", text=captured.stdout )
code_cell.outputs.append(output_stdout)
output_stderr = new_output(output_type="stream",
            name="stderr", text=captured.stderr )
code_cell.outputs.append(output_stderr)
# Process captured outputs
for output in captured.outputs:
    if hasattr(output, 'data'):
        # For rich outputs (e.g., plots), `data` attribute holds the MIME-typed representations
        output_cell = new_output(
            output_type='display_data',
            data=output.data,
            metadata=output.metadata
        )
    elif hasattr(output, 'text'):
        # For text outputs, `text` attribute holds the content
        output_cell = new_output(
            output_type='stream',
            name='stdout',  # or 'stderr' for error outputs
            text=output.text
        )
    else:
        continue  # Skip any unsupported output types

    # Append each output cell to the code cell
    code_cell.outputs.append(output_cell)
    
# code_cell = new_code_cell(source=code)
# # Add the output cell for the image
# output_cell = new_output(
#     output_type="display_data",
#     data={
#         "image/png": encoded_img_data
#     },
#     metadata={
#         "image/png": {
#             "width": "auto",
#             "height": "auto"
#         }
#     }
# )
# output_stdout = new_output(output_type="stream",
#             name="stdout",
#             text=captured.stdout
#             )
# output_stderr = new_output(output_type="stream",
#             name="stderr",
#             text=captured.stderr
#             )
# # Append the output to the code cell
# code_cell.outputs.append(output_cell)
# code_cell.outputs.append(output_stdout)
# code_cell.outputs.append(output_stderr)

nb.cells.append(code_cell)

# Write the notebook to a file
filename = 'notebook_with_plot.ipynb'
with open(filename, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

In [49]:
captured.stdout

''

In [40]:
captured

[]

In [52]:
captured.outputs

In [51]:
captured.outputs[0].data['image/png']

{'text/plain': '<Figure size 640x480 with 1 Axes>',
 'image/png': '...'}

In [57]:
captured.outputs[0]

AttributeError: 'RichOutput' object has no attribute 'text'

### Error handling

In [90]:
import nbformat
from nbformat.v4 import new_notebook, new_code_cell, new_output


code = "print('welcome!');import matplotlib.pyplot as plt\nplt.plot([1,2],[1,2,3,4])\nplt.show()"
out, captured, disp_images = python_code_exec(code)
# encoded_img_data = captured.outputs[0].data['image/png']

# Create a new notebook
nb = new_notebook()
# Add a code cell with the executed code
code_cell = new_code_cell(source=code)

if not out.success:
    if out.error_before_exec:
        output_stderr = new_output(output_type="stream",
                name="stderr", text=repr(out.error_before_exec) )
        code_cell.outputs.append(output_stderr)
    if out.error_in_exec:
        output_stderr = new_output(output_type="stream",
                name="stderr", text=repr(out.error_in_exec) )
        code_cell.outputs.append(output_stderr)

output_stdout = new_output(output_type="stream",
            name="stdout", text=captured.stdout )
code_cell.outputs.append(output_stdout)

output_stderr = new_output(output_type="stream",
            name="stderr", text=captured.stderr )
code_cell.outputs.append(output_stderr)
# Process captured outputs
for output in captured.outputs:
    if hasattr(output, 'data'):
        # For rich outputs (e.g., plots), `data` attribute holds the MIME-typed representations
        output_cell = new_output(
            output_type='display_data',
            data=output.data,
            metadata=output.metadata
        )
    elif hasattr(output, 'text'):
        # For text outputs, `text` attribute holds the content
        output_cell = new_output(
            output_type='stream',
            name='stdout',  # or 'stderr' for error outputs
            text=output.text
        )
    else:
        continue  # Skip any unsupported output types

    # Append each output cell to the code cell
    code_cell.outputs.append(output_cell)
    
nb.cells.append(code_cell)

# Write the notebook to a file
filename = 'notebook_with_plot_err.ipynb'
with open(filename, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

ValueError: x and y must have same first dimension, but have shapes (2,) and (4,)

In [83]:
out.error_in_exec

ValueError('x and y must have same first dimension, but have shapes (2,) and (4,)')

In [87]:
repr(out.error_in_exec)

"ValueError('x and y must have same first dimension, but have shapes (2,) and (4,)')"

### Pandas saving

In [94]:
captured.outputs[0].data

{'text/plain': '   a  b\n0  1  4\n1  2  5\n2  3  6',
 'text/html': '<div>\n<style scoped>\n    .dataframe tbody tr th:only-of-type {\n        vertical-align: middle;\n    }\n\n    .dataframe tbody tr th {\n        vertical-align: top;\n    }\n\n    .dataframe thead th {\n        text-align: right;\n    }\n</style>\n<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>a</th>\n      <th>b</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>1</td>\n      <td>4</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>2</td>\n      <td>5</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>3</td>\n      <td>6</td>\n    </tr>\n  </tbody>\n</table>\n</div>'}

In [91]:
import nbformat
from nbformat.v4 import new_notebook, new_code_cell, new_output


code = "import pandas as pd\ndf = pd.DataFrame({'a': [1,2,3], 'b': [4,5,6]})\ndf"
out, captured, disp_images = python_code_exec(code)
# encoded_img_data = captured.outputs[0].data['image/png']

# Create a new notebook
nb = new_notebook()
# Add a code cell with the executed code
code_cell = new_code_cell(source=code)
# handle errors
if not out.success:
    if out.error_before_exec:
        output_stderr = new_output(output_type="stream",
                name="stderr", text=repr(out.error_before_exec) )
        code_cell.outputs.append(output_stderr)
    if out.error_in_exec:
        output_stderr = new_output(output_type="stream",
                name="stderr", text=repr(out.error_in_exec) )
        code_cell.outputs.append(output_stderr)
# handle stdout outputs
output_stdout = new_output(output_type="stream",
            name="stdout", text=captured.stdout )
code_cell.outputs.append(output_stdout)
# handle stderror outputs
output_stderr = new_output(output_type="stream",
            name="stderr", text=captured.stderr )
code_cell.outputs.append(output_stderr)
# Process captured outputs
for output in captured.outputs:
    if hasattr(output, 'data'):
        # For rich outputs (e.g., plots), `data` attribute holds the MIME-typed representations
        output_cell = new_output(
            output_type='display_data',
            data=output.data,
            metadata=output.metadata
        )
    elif hasattr(output, 'text'):
        # For text outputs, `text` attribute holds the content
        output_cell = new_output(
            output_type='stream',
            name='stdout',  # or 'stderr' for error outputs
            text=output.text
        )
    else:
        continue  # Skip any unsupported output types

    # Append each output cell to the code cell
    code_cell.outputs.append(output_cell)
    
nb.cells.append(code_cell)

# Write the notebook to a file
filename = 'notebook_with_plot_df.ipynb'
with open(filename, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

## Final OVERALL example

In [95]:
import nbformat
from nbformat.v4 import new_notebook, new_code_cell, new_output

def create_code_cell_from_captured(code, out, captured):
    # Add a code cell with the executed code
    code_cell = new_code_cell(source=code)
    # handle errors
    if not out.success:
        if out.error_before_exec:
            output_stderr = new_output(output_type="stream",
                    name="stderr", text=repr(out.error_before_exec) )
            code_cell.outputs.append(output_stderr)
        if out.error_in_exec:
            output_stderr = new_output(output_type="stream",
                    name="stderr", text=repr(out.error_in_exec) )
            code_cell.outputs.append(output_stderr)
    # handle stdout outputs
    output_stdout = new_output(output_type="stream",
                name="stdout", text=captured.stdout )
    code_cell.outputs.append(output_stdout)
    # handle stderror outputs
    output_stderr = new_output(output_type="stream",
                name="stderr", text=captured.stderr )
    code_cell.outputs.append(output_stderr)
    # Process captured outputs
    for output in captured.outputs:
        if hasattr(output, 'data'):
            # For rich outputs (e.g., plots), `data` attribute holds the MIME-typed representations
            output_cell = new_output(
                output_type='display_data',
                data=output.data,
                metadata=output.metadata
            )
        elif hasattr(output, 'text'):
            # For text outputs, `text` attribute holds the content
            output_cell = new_output(
                output_type='stream',
                name='stdout',  # or 'stderr' for error outputs
                text=output.text
            )
        else:
            continue  # Skip any unsupported output types

        # Append each output cell to the code cell
        code_cell.outputs.append(output_cell)
    return code_cell

In [96]:

# encoded_img_data = captured.outputs[0].data['image/png']

# Create a new notebook
nb = new_notebook()
# Add a code cell with the executed code
code = "import pandas as pd\ndf = pd.DataFrame({'a': [1,2,3], 'b': [4,5,6]})\ndf"
out, captured, disp_images = python_code_exec(code)
# code_cell = create_code_cell_from_captured(code, out, captured)
nb.cells.append(create_code_cell_from_captured(code, out, captured))
code = "print('welcome!');import matplotlib.pyplot as plt\nplt.plot([1,2],[1,2,3,4])\nplt.show()"
out, captured, disp_images = python_code_exec(code)
nb.cells.append(create_code_cell_from_captured(code, out, captured))
code = "print('welcome!');import matplotlib.pyplot as plt\nplt.plot([1,2,3,4])\nplt.show()"
out, captured, disp_images = python_code_exec(code)
nb.cells.append(create_code_cell_from_captured(code, out, captured))

# Write the notebook to a file
filename = 'notebook_with_plot_all.ipynb'
with open(filename, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

ValueError: x and y must have same first dimension, but have shapes (2,) and (4,)

In [ ]:
nb.cells.extend([create_code_cell_from_captured(code, out, captured)]*3)

In [ ]:
filename = 'notebook_with_plot_all.ipynb'
with open(filename, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

/Users/binxuwang/miniforge3/envs/torch/lib/python3.11/site-packages/nbformat/__init__.py:129: DuplicateCellId: Non-unique cell id '146e522a' detected. Corrected to 'f488a10d'.
  validate(nb)


In [ ]:
code = "import matplotlib.pyplot as plt\nplt.plot([1,2,3,4])\nplt.show()"
out, captured, disp_images = python_code_exec(code)
# Store the code and its outputs
cells_data = [
    {
        "code": code,
        "outputs": [
            {
                "output_type": "stream",
                "name": "stdout",
                "text": captured.stdout
            },
            {
                "output_type": "stream",
                "name": "stderr",
                "text": captured.stderr
            }
        ]
    }
]